In [2]:
import pandas as pd


## agg 
语法
agg 是 aggregate 的别名，在指定轴上使用一个或多个操作进行聚合。建议使用 df.agg，而不是 df.aggregate。它的一般语法是：

DataFrame.agg(func=None, axis=0, *args, **kwargs)
参数
以下是参数说明

func : function, str, list 或者 dict。用于聚合数据的函数。如果是函数，则必须在传递给 DataFrame 或传递给 DataFrame.apply。可接受的组合为：
函数 function
字符串函数名 string function name
函数和/或函数名列表，如 [np.sum, 'mean']
轴标签->函数、函数名称或此类列表的字典
注：可不传，支持「嵌套重命名」（nested renamer）特殊语法，见下文介绍
axis : {0 or 'index', 1 or 'columns'}, 默认 0
如果 0 或者 'index':将函数应用于每列
如果 1 或者 'columns': 将函数应用于每一行
*args：要传递给“func”的位置参数
**kwargs：要传递给“func”的关键字参数
返回说明
返回的数据有 scalar（标量）, Series 或者 DataFrame，其中有：

scalar : 当 Series.agg 调用单个函数时
Series : 当 DataFrame.agg 调用单个函数时，Series.agg 调用多个函数时
DataFrame : 当 DataFrame.agg 调用多个函数时
其他说明
聚合操作始终在一个轴上执行，即索引轴（默认值）或列轴。这种行为不同于 numpy 的聚合函数（mean、median、prod、sum、std、var），其中默认值是计算展平数组（flattened array）的聚合，例如 numpy.mean(arr_2d) 而不是 numpy.mean(arr_2d, axis=0)。

传递变异对象（此处变异对象为非序列类型对象）的函数可能会产生意外行为或错误，因此不受支持，用户定义函数需要第一个参数传递一个 Series 以进行计算。

func 的参数可以为 None 不传，这种情况下，全关键字（可多个）形式传入，支持：

Series：支持 新索引名=聚合函数
DataFrame：支持 新列名=(原列名，聚合函数) 命名聚合功能（或者嵌套重命名），实现聚合后的列自定义列名功能，用 pd.NamedAgg 可显示定义
Grouby：也支持 新列名=(原列名，聚合函数) 命名聚合功能，同上

### Series.agg
* Series.agg 的应用和一般语法中介绍的一样，其中 axis 参数的取值为 {0 or ‘index’}，并且不可指定其他值，仅与 DataFrame.agg 做了兼容。

In [3]:
s = pd.Series([1, 2, 3, 4])
s

0    1
1    2
2    3
3    4
dtype: int64

In [4]:
s.agg('min')

1

In [5]:
s.agg(["min","max"])

min    1
max    4
dtype: int64

In [6]:
# 指定索引名
s.agg(A=max)

A    4
dtype: int64

In [7]:
s.agg(Big=max,Small=min)

Big      4
Small    1
dtype: int64

* Series 的 agg 不支持「嵌套重命名」（nested renamer）功能（关键字的值取元组，当然，这也没啥用）。

*******************************************************

### DataFrame.agg
| 常用示例

In [8]:
import numpy as np

In [9]:
df = pd.DataFrame([[1, 2, 3],
                    [4, 5, 6],
                    [7, 8, 9],
                    [np.nan, np.nan, np.nan]],
                   columns=['A', 'B', 'C'])

df

,A,B,C
0,1.0,2.0,3.0
1,4.0,5.0,6.0
2,7.0,8.0,9.0
3,NaN,NaN,NaN


In [10]:
# 在行上聚合这些函数
df.agg(['sum','min'])

,A,B,C
sum,12.0,15.0,18.0
min,1.0,2.0,3.0


In [11]:
df.agg({'A':['sum','min'],'B':['mean','max']})

,A,B
sum,12.0,NaN
min,1.0,NaN
mean,NaN,5.0
max,NaN,8.0


In [12]:
# 在列上聚合
df.agg("sum",axis=1)

0     6.0
1    15.0
2    24.0
3     0.0
dtype: float64

## 嵌套重命名
agg 支持「嵌套重命名」（nested renamer）特殊语法（原理见下文命名聚合 pd.NamedAgg 相关介绍，类似），如果无传入函数，而全为 关键字=(列名, 聚合函数) 形式会按重新标记处理。重新标记将关键字解析这对应的行索引，值取列名对应列的聚合函数计算结果。

In [13]:
# 在列上聚合不同的函数，并重命名结果数据帧的索引
df.agg(X=('A',max),Y=('B',min),Z=('C',np.mean))

,A,B,C
X,7.0,NaN,NaN
Y,NaN,2.0,NaN
Z,NaN,NaN,6.0


## apply


In [17]:
df

,A,B,C
0,1.0,2.0,3.0
1,4.0,5.0,6.0
2,7.0,8.0,9.0
3,NaN,NaN,NaN


In [22]:
df.apply(lambda x: x.B*2 if x.B is not None else "1",axis=1)


0     4.0
1    10.0
2    16.0
3     NaN
dtype: float64

0     4.0
1    10.0
2    16.0
3     NaN
dtype: float64

In [23]:
df.dtypes


A    float64
B    float64
C    float64
dtype: object

In [24]:
df.describe()


,A,B,C
count,3.0,3.0,3.0
mean,4.0,5.0,6.0
std,3.0,3.0,3.0
min,1.0,2.0,3.0
25%,2.5,3.5,4.5
50%,4.0,5.0,6.0
75%,5.5,6.5,7.5
max,7.0,8.0,9.0


In [27]:
df.columns

RangeIndex(start=0, stop=4, step=1)

In [28]:
df.index


RangeIndex(start=0, stop=4, step=1)